In [1]:
import re

class Patterns:
    URL_PATTERN=re.compile(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?\xab\xbb\u201c\u201d\u2018\u2019]))')
    HASHTAG_PATTERN = re.compile(r'(?P<hashtag>#\w*)')
    MENTION_PATTERN = re.compile(r'@\w*')
    RESERVED_WORDS_PATTERN = re.compile(r'^(RT|FAV)') # tweeter reserved words

    try:
        # UCS-4
        EMOJIS_PATTERN = re.compile(u'([\U00002600-\U000027BF])|([\U0001f300-\U0001f64F])|([\U0001f680-\U0001f6FF])')
    except re.error:
        # UCS-2
        EMOJIS_PATTERN = re.compile(u'([\u2600-\u27BF])|([\uD83C][\uDF00-\uDFFF])|([\uD83D][\uDC00-\uDE4F])|([\uD83D][\uDE80-\uDEFF])')

    SMILEYS_PATTERN = re.compile(r"(?:X|:|;|=)(?:-)?(?:\)|\(|O|D|P|S){1,}", re.IGNORECASE)
    NUMBERS_PATTERN = re.compile(r"(^|\s)(\-?\d+(?:\.\d)*|\d+)")

In [2]:
import tweepy
import datetime
from tweepy import Cursor

consumer_key = "RJu751Asy2vHaUaFSLGtGxWhx"
consumer_secret = "wvf4GLYEzP1Die5MRKd6DK1d0FCWiJKnMLJP1aQ7H28QBfIDRm" 
access_token = "1314603303796383746-hLWMOuVjFen5LOPQ7R600uJ6c38alm"
access_token_secret = "ZiLQteu4MuHxkKCjhaF0Oqt1znwDrkI37AdONbODVtFe8"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [3]:
def clean_tweets(rawData):
  cleanData = []
  for obj in rawData:
    obj = re.sub(Patterns.URL_PATTERN, "", obj)
    obj = re.sub(Patterns.HASHTAG_PATTERN, "", obj)
    obj = re.sub(Patterns.MENTION_PATTERN, "", obj)
    obj = re.sub(Patterns.RESERVED_WORDS_PATTERN, "", obj)
    obj = re.sub(Patterns.EMOJIS_PATTERN, "", obj)
    obj = re.sub(Patterns.EMOJIS_PATTERN, "", obj)
    obj = re.sub(Patterns.SMILEYS_PATTERN, "", obj)
    obj = re.sub(Patterns.NUMBERS_PATTERN , "", obj)
    obj = obj.rstrip("\n")
    obj = obj.replace("\n","")
    cleanData.append(obj)
  return cleanData

In [4]:
def extract_tweets(query):
    tweets=[]
    for status in Cursor(api.search,lang="en",
                     q=query, tweet_mode = 'extended', retweet_mode = 'extended', truncated=False).items(2500):
        tweets.append(status)
    return tweets

In [5]:
search_keywords = "Storm OR Winter OR Canada OR Temperature OR Flu OR Snow OR Indoor OR Safety"
tweets = extract_tweets(search_keywords)

In [6]:
rawTweets=[]
print(tweets[0].full_text)
for tweet in tweets:
    rawTweets.append(tweet.full_text) 


RT @tammyingram: And I posted this the other day, but while I was away on a fellowship, there was a hurricane in Charleston, so my moto boy…


In [7]:
cleanTweets = clean_tweets(rawTweets)
cleanTweets[0]

' : And I posted this the other day, but while I was away on a fellowship, there was a hurricane in Charleston, so my moto boy…'

In [8]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [11]:
bagOfWordsList = []
for tweet in cleanTweets:
    bagOfWords = {}
    words = word_tokenize(tweet)
    for word in words:
        if word in bagOfWords:
            bagOfWords[word] +=1
        else:
            bagOfWords[word] =1
    bagOfWordsList.append(bagOfWords)

In [12]:
positive_words = []
positiveFile = open("positive-words.txt","r")
words = positiveFile.readlines()
for word in words:
  word = word.rstrip("\n")
  positive_words.append(word)

In [13]:
negative_words = []
negativeFile = open("negative-words.txt","r")
words = negativeFile.readlines()
for word in words:
  word = word.rstrip("\n")
  negative_words.append(word)

In [14]:
finalData = []
i = 0
while( i < len(bagOfWordsList) ):
    bagOfWords = bagOfWordsList[i]
    row = {
        "Tweet": i+1,
        "Message": cleanTweets[i],
        "Matches": [],
        "Polarity": ""
    }
    posCount = 0
    negCount = 0
    posMatch = []
    negMatch = []
    for word, count in bagOfWords.items():
      if word in positive_words:
        posCount += 1*count
        posMatch.append(word)
      if word in negative_words:
        negCount += 1*count
        negMatch.append(word)
    res = posCount - negCount
    if (res > 0):
      row["Polarity"] = "Positive"
      row["Matches"] = posMatch
    elif (res < 0):
      row["Polarity"] = "Negative"
      row["Matches"] = negMatch
    elif (res == 0):
      row["Polarity"] = "Neutral"
      row["Matches"] = []
    finalData.append(row)
    i = i+1

In [15]:
import pandas as pd
tweet_analysis = pd.DataFrame(finalData)

In [16]:
tweet_analysis

,Tweet,Message,Matches,Polarity
0,1,": And I posted this the other day, but while ...",[],Neutral
1,2,I'm in Canada. I had to see the trailer the ...,[fans],Positive
2,3,: Boyz II Men &amp; Brian McKnight - Let It S...,[],Neutral
3,4,"Tbf I find it jokes how only,000 Millwall on t...","[rattle, flake]",Negative
4,5,Nope snow really bad like a snowicane!. Uz okie?,[],Neutral
...,...,...,...,...
2495,2496,: when your breath does the Thing in winter a...,[pretend],Negative
2496,2497,"Fellow New Englanders today, wishing you the b...","[best, luck]",Positive
2497,2498,: There's just a lot of liquid with this stor...,[],Neutral
2498,2499,"I've never technically left America, although ...",[],Neutral


In [17]:
tweet_analysis.to_csv("tweet_analysis.csv")